In [439]:
from gurobipy import *
import pandas as pd
import math

In [440]:
MAX_N = 96
MAX_T = 6
MAX_F = 3
LARGE_NUM = 10000000

In [441]:
file = pd.read_excel("客林專題資料.xlsx", sheet_name='商品相關參數')
f2 = pd.read_excel("客林專題資料.xlsx", sheet_name='非商品相關參數')

In [442]:
#Transfer from unit to package
file['存貨成本'] *= file['package']
file['BO缺貨成本'] *= file['package']
file['LS缺貨成本'] *= file['package']
file['快遞費用'] *= file['package']
file['空運費用'] *= file['package']
file['海運費用'] *= file['package']
file['進貨價格(TWD)'] *= file['package']

# demand / package
file['demand_t'] /= file['package']
file['demand_t+1'] /= file['package']
file['demand_t+2'] /= file['package']
file['demand_t+3'] /= file['package']
file['demand_t+4'] /= file['package']
file['demand_t+5'] /= file['package']

# In-transit inventory / ending inventory / package
file['在途存貨_t+1'] /= file['package']
file['在途存貨_t+2'] /= file['package']
file['期末存貨_t-1'] /= file['package']

In [444]:
cols = ['商品', 'package', 'backorder percentage', '在途存貨_t+1', '在途存貨_t+2','期末存貨_t-1', 'demand_t', 'demand_t+1', 'demand_t+2', 'demand_t+3','demand_t+4', 'demand_t+5', '存貨成本', 'BO缺貨成本', 'LS缺貨成本', '快遞費用','空運費用',  '海運費用', '可快遞', '可空運', '可海運', '進貨價格(TWD)']
file = file[cols]
ren_col = {'商品':'i', 'package':'P', 'backorder percentage':'beta', '在途存貨_t+1':'Q_t+1', '在途存貨_t+2':'Q_t+2','期末存貨_t-1':'I', 'demand_t':'D_t', 'demand_t+1':'D_t+1', 'demand_t+2':"D_t+2", 'demand_t+3':"D_t+3",'demand_t+4':"D_t+4", 'demand_t+5':"D_t+5", '存貨成本':'Co', 'BO缺貨成本':'Cb', 'LS缺貨成本':'Cl', '快遞費用':'G1','空運費用':"G2",  '海運費用':'G3', '可快遞':"R1", '可空運':"R2", '可海運':"R3", '進貨價格(TWD)':"m"}
file = file.rename(columns=ren_col)

In [445]:
K1 = f2.iloc[0, 1]
K2 = f2.iloc[0, 3]
K3 = f2.iloc[0, 5]
K = [0, K1, K2, K3]

In [446]:
#variable cost of shipping fee
G = []
G.append([])
for i in range(MAX_N):
    G.append([])
    G[i + 1].append(0)
    for f in range(MAX_F):
        G[i + 1].append(file.iloc[i, 15+f])
        
#Demand        
D = []
D.append([])
for t in range(MAX_T):
    D.append([])
    D[t + 1].append(0)
    for i in range(MAX_N):
        D[t + 1].append(file.iloc[i, 6+t])

#In-transit inventory       
Q = []
Q.append([])
Q.append([])
for t in range(1, 3):
    Q.append([])
    Q[t + 1].append(0)
    for i in range(MAX_N):
        Q[t + 1].append(file.iloc[i, 2+t])

#Purchase price
M = []
M.append(0)
for i in range(MAX_N):
    M.append(file.iloc[i, 21])
    
#inventory cost
Co = []
Co.append(0)
for i in range(MAX_N):
    Co.append(file.iloc[i, 12])
    
#lost sales cost
Cb = []
Cb.append(0)
for i in range(MAX_N):
    Cb.append(file.iloc[i, 13])
    
#backorder cost
Cl = []
Cl.append(0)
for i in range(MAX_N):
    Cl.append(file.iloc[i, 14])
    
#backorder percentage
beta = []
beta.append(0)
for i in range(MAX_N):
    beta.append(file.iloc[i, 2])
    
#Initial inventory
I0 = []
I0.append(0)
for i in range(MAX_N):
    I0.append(file.iloc[i, 5])
    
#shipping method
R = []
R.append([])
for i in range(MAX_N):
    R.append([])
    R[i + 1].append(0)
    for f in range(MAX_F):
        R[i + 1].append(file.iloc[i, 18+f])

In [447]:
#Initialization of list
x = [[[0]*(MAX_F+1) for i in range(MAX_N+1)] for j in range(MAX_T+1)]
y = [[0]*(MAX_N+1) for i in range(MAX_T+1)]
d = [[0]*(MAX_N+1) for i in range(MAX_T+1)]
#order quantity of all items
q = [0] * (MAX_N+1)

In [448]:
maxX = []
maxX.append(0)
for i in range(1, MAX_N + 1):
    tmp = 0
    for t in range(1, MAX_T + 1):
        tmp += D[t][i]
    for t in range(2, 4):
        tmp -= Q[t][i]
    tmp -= I0[i] + 1
    maxX.append(max(round(tmp, 0), 0))

In [449]:
print(maxX)

[0, 0, 0, 2.0, 0, 1.0, 1.0, -0.0, 0, 8.0, 0, 0, 0, 0, 13.0, 0, 0, 25.0, 12.0, 0, 0, 0, 0, 0, 0, 0, 0, 18.0, 0, 0, 0, 0, 0, 49.0, 0, 0, 59.0, 0, 0, 0, 23.0, 9.0, 0, 0, 0, 0, 0, 0, 0, 17.0, 0, 0, 66.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 12.0, 0, 0, 0, 0, 0, 0, 487.0, 0, 34.0, 40.0, 0, 0, 50.0, 1.0, 0, 0, 0, 125.0, 573.0, 0, 0, 0, 93.0, 0, 8.0, 0, 224.0]


In [450]:
for t in range(1, MAX_T+1):
    for i in range(1, MAX_N+1):
        #t=1
        if t == 1:
            d[t][i] = max(0, -(I0[i] - D[t][i]))
            y[t][i] = max(0, I0[i] - D[t][i])
        elif t == 2 or t == 3:
            d[t][i] = max(0, -(y[t-1][i] - D[t][i] + Q[t][i]))
            y[t][i] = max(0, y[t-1][i] - D[t][i] + Q[t][i])
        else:
            d[t][i] = max(0, -(y[t-1][i] - D[t][i]))
            y[t][i] = max(0, y[t-1][i] - D[t][i])
            
for i in range(1, MAX_N + 1):
    if d[1][i] > 0:
        d[2][i] += d[1][i] * beta[i]

In [451]:
for i in range(1, MAX_N+1):
    print(i, ' ', end='')
    for t in range(1, MAX_T+1):
        d[t][i] = int(round(d[t][i]))
        print(d[t][i], ' ', end='')
    print('')

1  0  0  0  0  0  0  
2  0  0  0  0  0  0  
3  0  0  0  0  0  3  
4  0  0  0  0  0  0  
5  0  0  0  0  1  1  
6  0  0  0  1  1  1  
7  0  0  0  0  0  1  
8  0  0  0  0  0  0  
9  0  0  0  0  3  6  
10  0  0  0  0  0  0  
11  0  0  0  0  0  0  
12  0  0  0  0  0  0  
13  0  0  0  0  0  0  
14  0  0  0  0  7  8  
15  0  0  0  0  0  0  
16  0  0  0  0  0  0  
17  4  3  5  5  5  5  
18  0  0  0  0  7  6  
19  0  0  0  0  0  0  
20  0  0  0  0  0  0  
21  0  0  0  0  0  0  
22  0  0  0  0  0  0  
23  0  0  0  0  0  0  
24  0  0  0  0  0  0  
25  0  0  0  0  0  0  
26  0  0  0  0  0  0  
27  0  0  0  0  7  12  
28  0  0  0  0  0  0  
29  0  0  0  0  0  0  
30  0  0  0  0  0  0  
31  0  0  0  0  0  0  
32  0  0  0  0  0  0  
33  10  13  10  10  10  0  
34  0  10  0  0  0  0  
35  43  15  0  0  0  0  
36  8  19  16  10  10  0  
37  0  0  0  0  0  0  
38  0  18  0  0  0  0  
39  0  0  0  0  0  0  
40  0  0  0  2  10  12  
41  0  4  2  2  2  0  
42  0  0  0  0  0  0  
43  0  0  0  0  0  0  
44  

In [452]:
fb = [0] * 7
fb[2] = 1
for t in range(3, MAX_T + 1):
    min_cos = 999999999
    for f in range(1, min(4, t)):
        cos = K[f]
        for i in range(1, MAX_N + 1):
            cos += d[t][i] * (M[i] + G[i][f])
        if cos < min_cos:
            min_cos = cos
            fb[t] = f

In [453]:
print(fb)

[0, 0, 1, 2, 3, 2, 3]


In [454]:
for i in range(1, MAX_N+1):
    for t in range(1, MAX_T+1):
        if d[t][i] > 0:
            if R[i][f] == 1:
                x[t - fb[t]][i][fb[t]] += d[t][i]
            else:
                if R[i][1] == 1:
                    x[t - 1][i][1] += d[t][i]
                elif R[i][2] == 1:
                    x[t - 2][i][1] += d[t][i]
                else:
                    x[t - 3][i][3] += d[t][i]

In [455]:
for t in range(1, MAX_T+1):
    for i in range(1, MAX_N+1):
        for f in range(1, MAX_F+1):
            print('x'+str(t)+','+str(i)+','+str(f)+' = '+str(x[t][i][f]))

x1,1,1 = 0
x1,1,2 = 0
x1,1,3 = 0
x1,2,1 = 0
x1,2,2 = 0
x1,2,3 = 0
x1,3,1 = 0
x1,3,2 = 0
x1,3,3 = 0
x1,4,1 = 0
x1,4,2 = 0
x1,4,3 = 0
x1,5,1 = 0
x1,5,2 = 0
x1,5,3 = 0
x1,6,1 = 0
x1,6,2 = 0
x1,6,3 = 1
x1,7,1 = 0
x1,7,2 = 0
x1,7,3 = 0
x1,8,1 = 0
x1,8,2 = 0
x1,8,3 = 0
x1,9,1 = 0
x1,9,2 = 0
x1,9,3 = 0
x1,10,1 = 0
x1,10,2 = 0
x1,10,3 = 0
x1,11,1 = 0
x1,11,2 = 0
x1,11,3 = 0
x1,12,1 = 0
x1,12,2 = 0
x1,12,3 = 0
x1,13,1 = 0
x1,13,2 = 0
x1,13,3 = 0
x1,14,1 = 0
x1,14,2 = 0
x1,14,3 = 0
x1,15,1 = 0
x1,15,2 = 0
x1,15,3 = 0
x1,16,1 = 0
x1,16,2 = 0
x1,16,3 = 0
x1,17,1 = 3
x1,17,2 = 5
x1,17,3 = 5
x1,18,1 = 0
x1,18,2 = 0
x1,18,3 = 0
x1,19,1 = 0
x1,19,2 = 0
x1,19,3 = 0
x1,20,1 = 0
x1,20,2 = 0
x1,20,3 = 0
x1,21,1 = 0
x1,21,2 = 0
x1,21,3 = 0
x1,22,1 = 0
x1,22,2 = 0
x1,22,3 = 0
x1,23,1 = 0
x1,23,2 = 0
x1,23,3 = 0
x1,24,1 = 0
x1,24,2 = 0
x1,24,3 = 0
x1,25,1 = 0
x1,25,2 = 0
x1,25,3 = 0
x1,26,1 = 0
x1,26,2 = 0
x1,26,3 = 0
x1,27,1 = 0
x1,27,2 = 0
x1,27,3 = 0
x1,28,1 = 0
x1,28,2 = 0
x1,28,3 = 0
x1,29,1 = 0
x1,29,2

x4,39,3 = 0
x4,40,1 = 0
x4,40,2 = 0
x4,40,3 = 0
x4,41,1 = 0
x4,41,2 = 0
x4,41,3 = 0
x4,42,1 = 0
x4,42,2 = 0
x4,42,3 = 0
x4,43,1 = 0
x4,43,2 = 0
x4,43,3 = 0
x4,44,1 = 0
x4,44,2 = 0
x4,44,3 = 0
x4,45,1 = 0
x4,45,2 = 0
x4,45,3 = 0
x4,46,1 = 0
x4,46,2 = 0
x4,46,3 = 0
x4,47,1 = 0
x4,47,2 = 0
x4,47,3 = 0
x4,48,1 = 0
x4,48,2 = 0
x4,48,3 = 0
x4,49,1 = 0
x4,49,2 = 0
x4,49,3 = 0
x4,50,1 = 0
x4,50,2 = 0
x4,50,3 = 0
x4,51,1 = 0
x4,51,2 = 0
x4,51,3 = 0
x4,52,1 = 0
x4,52,2 = 0
x4,52,3 = 0
x4,53,1 = 0
x4,53,2 = 0
x4,53,3 = 0
x4,54,1 = 0
x4,54,2 = 0
x4,54,3 = 0
x4,55,1 = 0
x4,55,2 = 0
x4,55,3 = 0
x4,56,1 = 0
x4,56,2 = 0
x4,56,3 = 0
x4,57,1 = 0
x4,57,2 = 0
x4,57,3 = 0
x4,58,1 = 0
x4,58,2 = 0
x4,58,3 = 0
x4,59,1 = 0
x4,59,2 = 0
x4,59,3 = 0
x4,60,1 = 0
x4,60,2 = 0
x4,60,3 = 0
x4,61,1 = 0
x4,61,2 = 0
x4,61,3 = 0
x4,62,1 = 0
x4,62,2 = 0
x4,62,3 = 0
x4,63,1 = 0
x4,63,2 = 0
x4,63,3 = 0
x4,64,1 = 0
x4,64,2 = 0
x4,64,3 = 0
x4,65,1 = 0
x4,65,2 = 0
x4,65,3 = 0
x4,66,1 = 0
x4,66,2 = 0
x4,66,3 = 0
x4,67,1 = 0
x4,6

In [456]:
def cost(x):
    prod_info = pd.read_excel('客林專題資料.xlsx', '商品相關參數')
    P = prod_info['package']
    
    I0 = [0] + list(prod_info['期末存貨_t-1'] / P)
    Q1 = [0] + list(prod_info['在途存貨_t+1'] / P)
    Q2 = [0] + list(prod_info['在途存貨_t+2'] / P)
    D  = []
    for t in range(MAX_T+1):
        item_list = []
        for i in range(MAX_N+1):
            item_list.append(0)
        D.append(item_list)
    for t in range(1, MAX_T+1):
        row_name = 'demand_t+' + str(t-1) if t >= 2 else 'demand_t'
        D[t] = [0] + list(prod_info[row_name] / P)
    
    v  = []
    for t in range(MAX_T+1):
        item_list = []
        for i in range(MAX_N+1):
            item_list.append(-1)
        v.append(item_list)
    
    B  = [0] + list(prod_info['backorder percentage'])
    
    total_cost = int()
    # calculate inventory cost
    inventory_cost = int()
    # calculate each term's(t) ending inventories(i) y[t][i]
    y = []
    for t in range(MAX_T+1):
        item_list = []
        for i in range(MAX_N+1):
            item_list.append(0)
        y.append(item_list)
    y[0] = I0
    for a in range(1, len(v[0])):
        if y[0][a] >= 0:
            v[0][a] = 1
        else:
            v[0][a] = 0
    for t in range(1, MAX_T+1):
        for i in range(1, MAX_N+1):
            # 期初存貨 - 需求 + 在途存貨 + 進貨量
            # 期初存貨
            beg_inv = v[t-1][i] * y[t-1][i] + (1 - v[t-1][i]) * (B[i] * y[t-1][i])
            y[t][i] = beg_inv
            # 需求
            y[t][i] -= D[t][i]
            # 在途存貨
            if (t == 2):
                y[t][i] += Q1[i]
            elif (t == 3):
                y[t][i] += Q2[i]
            # 進貨量
            for f in range(t-1):
                y[t][i] += x[t-1-f][i][f+1]
                if (f == 2):
                    break
            v[t][i] = 1 if y[t][i] >= 0 else 0
            
    Co = [0] + list(prod_info['存貨成本'] * P)
    for t in range(1, MAX_T+1):
        for i in range(1, MAX_N+1):
            inventory_cost += v[t][i] * Co[i] * y[t][i]
    
    # calculate the ordering cost
    ordering_cost = int()
    # 變動成本
    M  = [0] + list(prod_info['進貨價格(TWD)'] * P)
    C1 = [0] + list(prod_info['快遞費用'] * P)
    C2 = [0] + list(prod_info['空運費用'] * P)
    C3 = [0] + list(prod_info['海運費用'] * P)
    G  = []

    for i in range(MAX_N+1):
        freight_cost = [0, C1[i], C2[i], C3[i]]
        G.append(freight_cost)

    for t in range(1, MAX_T+1):
        for i in range(1, MAX_N+1):
            for f in range(1, MAX_F+1):
                ordering_cost += (M[i] + G[i][f]) * x[t][i][f]

    # 固定成本
    nonprod_info = pd.read_excel('客林專題資料.xlsx', '非商品相關參數')
    
    K = []
    K.append(0)
    K.append(float(nonprod_info['快遞固定成本']))
    K.append(float(nonprod_info['空運固定成本']))
    K.append(float(nonprod_info['海運固定成本']))
    
    w = []
    for t in range(MAX_T+1):
        freight_list = []
        for f in range(MAX_F+1):
            freight_list.append(False)
        w.append(freight_list)
    for f in range(1, MAX_F+1):
        for t in range(1, MAX_T+1):
            for i in range(1, MAX_N+1):
                if (x[t][i][f] > 0):
                    w[t][f] = True
                    break
    
    for t in range(1, MAX_T+1):
        for f in range(1, MAX_F+1):
            ordering_cost += K[f] * w[t][f]
            
    # calculate lost sale cost
    lostsales_cost = int()
    Cb = [0] + list(prod_info['BO缺貨成本'] * P)
    Cl = [0] + list(prod_info['LS缺貨成本'] * P)
    for t in range(1, MAX_T+1):
        for i in range(1, MAX_N+1):
            lostsales_cost += (1-v[t][i]) * (-B[i]*Cb[i]*y[t][i] - (1-B[i])*Cl[i]*y[t][i])
    
    total_cost = ordering_cost + inventory_cost + lostsales_cost
    print(total_cost)
    return total_cost

In [457]:
cost(x)

14000672.93496766


14000672.93496766

In [458]:
result = pd.DataFrame(columns = ['訂購時間', '訂購商品', '訂購數量', '運送方式'])
for t in range(1, MAX_T+1):
    for i in range(1, MAX_N+1):
        for f in range(1, MAX_F+1):
            if x[t][i][f] > 0:
                shipTran = {"1":'快遞', "2":'空運', "3":'海運'}
                a = pd.Series([t, i, x[t][i][f], shipTran[str(f)]], index=['訂購時間', '訂購商品', '訂購數量', '運送方式'])
                result = result.append(a, ignore_index=True)

In [459]:
import os
path = os.getcwd()
result.to_excel(path+"/存貨成本變成0.xlsx")